In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
class ResidualUnit(keras.layers.Layer):

    def __init__ (self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, 3, strides=strides, padding='same', use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, strides=1, padding='same', use_bias=False),
            keras.layers.BatchNormalization()
        ]

        self.skip_layers = []

        # If stride is greater than 1 we need to reduce the size of the input as well, before we can add it to the output
        # of the main layers
        if strides > 1 :
            self.skip_layers = [
                keras.layers.Conv2D(filters, 1, strides=strides, padding='same', use_bias=False),
                keras.layers.BatchNormalization()
            ]
    
    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(Z)
        
        return self.activation(Z + skip_Z)

In [4]:
resnet_34 = keras.models.Sequential()

resnet_34.add(keras.layers.Conv2D(64, 7, strides=2, padding='same', use_bias=False, input_shape=[224,224,3]))
resnet_34.add(keras.layers.BatchNormalization())
resnet_34.add(keras.layers.Activation('relu'))
resnet_34.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))

prev_filter = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filter else 2 # when switching filter numbers we want to increase the stride to 2
    resnet_34.add(ResidualUnit(filters, strides=strides))
    prev_filter = filters

resnet_34.add(keras.layers.GlobalAveragePooling2D())
resnet_34.add(keras.layers.Flatten())
resnet_34.add(keras.layers.Dense(1000, activation='softmax')) # ImageNet has 1000 classes

2022-07-25 21:10:29.022462: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-25 21:10:29.023203: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

